In [19]:
import requests
import json
import requests
from bs4 import BeautifulSoup
import os


# create the directory if it doesn't already exist
if not os.path.exists('News-Articles'):
    os.makedirs('News-Articles')


# Load URLs from JSON file
with open('urls.json', 'r') as f:
    urls = json.load(f)

for url in urls:
    url_string = url["url"]
    r = requests.get(url_string)

    if r.status_code == 200:
        soup = BeautifulSoup(r.content, "html.parser")
        article = soup.find("article")
        # Get the text out of the soup and print it
        content = article.get_text()
        # print(content)
        
        # construct the path to the file within the News-Articles directory
        filename = os.path.join('News-Articles', url['title'] + '.txt')
        with open(filename, 'a') as f:
            f.write(content)
            f.write('\n\n')
        print(f'Successfully saved {filename}')
    else:
        print(f"Error: {r.status_code}")


Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/business.txt
Successfully saved News-Articles/business.txt
Successfully saved News-Articles/business.txt
Successfully saved News-Articles/technology.txt
Successfully saved News-Articles/technology.txt
Successfully saved News-Articles/technology.txt
Successfully saved News-Articles/technology.txt
Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/sports.txt
Successfully saved News-Articles/politics.txt
Successfully saved News-Articles/politics.txt
Successfully saved News-Articles/politics.txt
Successfully saved News-Articles/politics.txt
Successfully saved News-Articles/health.txt
Successfully saved News-Articles/health.txt
Successfully saved News-Articles/health.txt
Successfully saved News-Articles/health.txt
Successfully saved News-Articles/climate.txt
Successfully saved News-Articles/climate.txt


In [20]:
import csv

# Directory where the .txt files are located
directory = './News-Articles/'

# Create a list to store the rows of the CSV file
csv_rows = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        # Read the contents of the file
        with open(os.path.join(directory, filename), 'r') as f:
            content = f.read().replace('\n', ' ')
        
        # Get the title from the filename
        title = filename.split('.')[0]
        
        # Append the title and content to the CSV row list
        csv_rows.append([title, content])

# Write the CSV file
with open('news_corpus.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['Topic', 'Content'])  # write the header row
    writer.writerows(csv_rows)  # write the content rows
    print("Successfully created news_corpus.csv")


Successfully created news_corpus.csv


In [28]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Load the news corpus into a Pandas DataFrame
df = pd.read_csv('news_articles.csv')

# Initialize NLTK components
stop_words = set(stopwords.words('english'))
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Function to preprocess text data
def preprocess_text(Content):
    # Remove punctuation
    Content = Content.translate(str.maketrans('', '', string.punctuation))
    
    # Remove digits
    Content = re.sub(r'\d+', '', Content)
    
    # Tokenize the text into words
    words = word_tokenize(Content.lower())

    # Remove stop words
    words = [w for w in words if not w in stop_words]

    # Lemmatize the remaining words
    words = [lemmatizer.lemmatize(w) for w in words]

    # Rejoin the words into a single string
    return ' '.join(words)

# Apply the preprocessing function to the news corpus and save into a new CSV file
df['Content'] = df['Content'].apply(preprocess_text)
df.to_csv('preprocessed_news_corpus.csv', index=False)

print(df)

[nltk_data] Downloading package wordnet to /home/silver/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


         Topic                                            Content
0       sports  manchester united premier league club owe almo...
1       sports  felix jones england recruit exireland fullback...
2       sports  premier league tightens ownership rule stop hu...
3     business  happening house price could crashpublished hou...
4     business  india middle class hit rising cost livingpubli...
5     business  bank england heightened alert banking turmoilp...
6   technology  elon musk among expert urging halt ai training...
7   technology  elon musk twitter bos announces blue tick shak...
8   technology  fun apps banned french official phonespublishe...
9   technology  google face new multibillion advertising lawsu...
10      sports  man city v liverpool danny murphy jurgen klopp...
11      sports  premier league relegation nine team danger dro...
12    politics  donald trump indictment exus president charged...
13    politics  humza yousafs first fmqs hit series disruption...
14    poli

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



# Load the preprocessed news corpus
df = pd.read_csv('preprocessed_news_corpus.csv')

# Create a TF-IDF Vectorizer object
vectorizer = TfidfVectorizer()

# vectorizer = TfidfVectorizer(ngram_range=(1, 2))
# vectorizer = TfidfVectorizer(ngram_range=(2, 3))
# vectorizer = TfidfVectorizer(max_df=0.5)
vectorizer = TfidfVectorizer(min_df=4)


# Vectorize the text data
X = vectorizer.fit_transform(df['Content'])

# Get the labels
y = df['Topic']

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predict the categories for the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))



Accuracy: 0.7142857142857143
Precision: 0.619047619047619
Recall: 0.7142857142857143
F1-Score: 0.6428571428571429


/home/silver/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(SOLVED)
Getting an accuracy, precision, recall, and F1-score of 0.0 meanig that the classifier did not correctly predict any of the classes in the test set.
I havent figured why since adjusting the parameters of the TF-IDF Vectorizer to improve the quality of the features did not make any difference.
Let me try using Bag-of-Words (BoW) model with CountVectorizer instead and see how it turns out. 

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the preprocessed news corpus
df = pd.read_csv('preprocessed_news_corpus.csv')

# Create a Count Vectorizer object
vectorizer = CountVectorizer()

# Vectorize the text data
X = vectorizer.fit_transform(df['Content'])

# Get the labels
y = df['Topic']

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predict the categories for the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))


Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


In [33]:
import pandas as pd

# Load the preprocessed news corpus
df = pd.read_csv('preprocessed_news_corpus.csv')

# check for class imbalance
df['Topic'].value_counts()


sports        7
climate       7
business      6
technology    6
politics      5
health        4
Name: Topic, dtype: int64

(SOLVED)
This was the issue all along. my corpus is imbalanced. fixable by increasing the size of my dataset by adding more instances of each category.

In [35]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the preprocessed news corpus
df = pd.read_csv('preprocessed_news_corpus.csv')

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Vectorize the text data
X = vectorizer.fit_transform(df['Content'])

# Get the labels
y = df['Topic']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=150, random_state=42)
clf.fit(X_train, y_train)

# Predict the categories for the test set
y_pred = clf.predict(X_test)

# Evaluate the performance of the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1-Score:", f1_score(y_test, y_pred, average='weighted'))


Accuracy: 0.5714285714285714
Precision: 0.7857142857142857
Recall: 0.5714285714285714
F1-Score: 0.6285714285714287


/home/silver/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/silver/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
